In [1]:
GPU_device_id = str(0)
model_id_save_as = 'learningcurve-cnn-easy-final-reluupdate'
# architecture_id = '../hyperparameter_search/hyperparameter-search-results/CNN-kfoldseasy-final-1_14'
architecture_id = '../hyperparameter_search/hyperparameter-search-results/CNN-kfoldseasy-final-1-reluupdate_33'
model_class_id = 'CNN1D'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_200keV_log10time_10000.npy'
difficulty_setting = 'easy'

train_sizes = [10000, 50, 100, 500, 1000, 5000, 15000, 20000, ]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [3]:
from annsa.model_classes import build_cnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [6]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [7]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [4]:
model_features = load_features(architecture_id)
model_features.output_function = tf.nn.softmax
model_features.cnn_kernels = model_features.cnn_kernel
model_features.pool_sizes = model_features.pool_size
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.metrics = [f1]
model_features.dropout_rate = model_features.dropout_probability
model_features.output_function = tf.nn.softmax
model_features.input_dim = 1024

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model = compile_model(
            build_cnn_model,
            model_features)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1024, 1)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1024, 4)           68        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 512, 4)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 8)            520       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 256, 8)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 256, 16)           2064      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 128, 16)           0         
__________

## Train network

# Scale input data

In [11]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [5]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [13]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        model = compile_model(
            build_cnn_model,
            model_features)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 10000
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 10000 samples, validate on 3000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/2000
10000/10000 [==============================] - 4s 386us/sample - loss: 1.5050 - f1: 0.6233 - val_loss: 0.6766 - val_f1: 0.8853
Epoch 2/2000
10000/10000 [==============================] - 2s 219us/sample - loss: 0.5624 - f1: 0.8951 - val_loss: 0.4881 - val_f1: 0.9122
Epoch 3/2000
10000/10000 [==============================] - 2s 219us/sample - loss: 0.4416 - f1: 0.9137 - val_loss: 0.4302 - val_f1: 0.9099
Epoch 4/2000
10000/10000 [==============================] - 2s 219us/sample - loss: 0.3943 - f1: 0.9242 - val_loss: 0.3405 - val_f1: 0.9450
Epoch 5/2000
10000/10000 [==============================] - 2s 219us/sample - loss: 0.3461 - f1: 0.9372 - val_loss: 0.3634 - val_f1: 0.9290
Epoch 6/2000
100

Epoch 7/2000
10000/10000 [==============================] - 2s 221us/sample - loss: 0.2881 - f1: 0.9504 - val_loss: 0.3426 - val_f1: 0.9469
Epoch 8/2000
10000/10000 [==============================] - 2s 221us/sample - loss: 0.2620 - f1: 0.9578 - val_loss: 0.2450 - val_f1: 0.9587
Epoch 9/2000
10000/10000 [==============================] - 2s 221us/sample - loss: 0.2309 - f1: 0.9629 - val_loss: 0.2408 - val_f1: 0.9510
Epoch 10/2000
10000/10000 [==============================] - 2s 221us/sample - loss: 0.2789 - f1: 0.9563 - val_loss: 0.2641 - val_f1: 0.9505
Epoch 11/2000
10000/10000 [==============================] - 2s 220us/sample - loss: 0.2223 - f1: 0.9668 - val_loss: 0.2206 - val_f1: 0.9629
Epoch 12/2000
10000/10000 [==============================] - 2s 221us/sample - loss: 0.1998 - f1: 0.9733 - val_loss: 0.1754 - val_f1: 0.9802
Epoch 13/2000
10000/10000 [==============================] - 2s 221us/sample - loss: 0.2282 - f1: 0.9676 - val_loss: 0.1992 - val_f1: 0.9771
Epoch 14/2000
10

Epoch 44/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.8836 - f1: 0.8951 - val_loss: 2.0138 - val_f1: 0.4668
Epoch 45/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.8634 - f1: 0.9189 - val_loss: 2.0132 - val_f1: 0.4630
Epoch 46/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.8394 - f1: 0.9122 - val_loss: 2.0086 - val_f1: 0.4756
Epoch 47/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.8371 - f1: 0.9483 - val_loss: 2.0033 - val_f1: 0.4796
Epoch 48/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.8026 - f1: 0.9384 - val_loss: 1.9925 - val_f1: 0.4800
Epoch 49/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.7829 - f1: 0.9611 - val_loss: 1.9942 - val_f1: 0.4819
Epoch 50/2000
50/50 [==============================] - 0s 5ms/sample - loss: 0.7624 - f1: 0.9611 - val_loss: 1.9818 - val_f1: 0.4885
Epoch 51/2000
50/50 [==============================] - 0s 5ms/sample 

...

1000/1000 [==============================] - 1s 522us/sample - loss: 0.3367 - f1: 0.9501 - val_loss: 0.3789 - val_f1: 0.9287
Epoch 36/2000
1000/1000 [==============================] - 1s 522us/sample - loss: 0.3187 - f1: 0.9594 - val_loss: 0.3929 - val_f1: 0.9197
Epoch 37/2000
1000/1000 [==============================] - 1s 521us/sample - loss: 0.3138 - f1: 0.9600 - val_loss: 0.4348 - val_f1: 0.9167
Epoch 38/2000
1000/1000 [==============================] - 1s 520us/sample - loss: 0.3348 - f1: 0.9522 - val_loss: 0.4228 - val_f1: 0.9197
Epoch 39/2000
1000/1000 [==============================] - 1s 520us/sample - loss: 0.3328 - f1: 0.9509 - val_loss: 0.3844 - val_f1: 0.9252
Epoch 40/2000
1000/1000 [==============================] - 1s 520us/sample - loss: 0.3566 - f1: 0.9435 - val_loss: 0.3827 - val_f1: 0.9358
Epoch 41/2000
1000/1000 [==============================] - 1s 517us/sample - loss: 0.3320 - f1: 0.9516 - val_loss: 0.4334 - val_f1: 0.9043
Epoch 42/2000
1000/1000 [================

Epoch 24/2000
5000/5000 [==============================] - 2s 303us/sample - loss: 0.2157 - f1: 0.9773 - val_loss: 0.1984 - val_f1: 0.9786
Epoch 25/2000
5000/5000 [==============================] - 2s 304us/sample - loss: 0.1956 - f1: 0.9827 - val_loss: 0.1860 - val_f1: 0.9847
Epoch 26/2000
5000/5000 [==============================] - 2s 304us/sample - loss: 0.1683 - f1: 0.9885 - val_loss: 0.1720 - val_f1: 0.9850
Epoch 27/2000
5000/5000 [==============================] - 2s 304us/sample - loss: 0.2436 - f1: 0.9774 - val_loss: 0.2216 - val_f1: 0.9865
Epoch 28/2000
5000/5000 [==============================] - 2s 304us/sample - loss: 0.1803 - f1: 0.9891 - val_loss: 0.2299 - val_f1: 0.9684
Epoch 29/2000
5000/5000 [==============================] - 2s 305us/sample - loss: 0.2328 - f1: 0.9827 - val_loss: 0.1841 - val_f1: 0.9903
Epoch 30/2000
5000/5000 [==============================] - 2s 304us/sample - loss: 0.1734 - f1: 0.9907 - val_loss: 0.1705 - val_f1: 0.9902
Epoch 31/2000
5000/5000 [==

5000/5000 [==============================] - 2s 310us/sample - loss: 0.3016 - f1: 0.9505 - val_loss: 0.3302 - val_f1: 0.9391
Epoch 12/2000
5000/5000 [==============================] - 2s 307us/sample - loss: 0.3208 - f1: 0.9463 - val_loss: 0.2899 - val_f1: 0.9458
Epoch 13/2000
5000/5000 [==============================] - 2s 304us/sample - loss: 0.2800 - f1: 0.9525 - val_loss: 0.3186 - val_f1: 0.9375
Epoch 14/2000
5000/5000 [==============================] - 2s 307us/sample - loss: 0.3078 - f1: 0.9507 - val_loss: 0.2671 - val_f1: 0.9529
Epoch 15/2000
5000/5000 [==============================] - 2s 308us/sample - loss: 0.3035 - f1: 0.9487 - val_loss: 0.2564 - val_f1: 0.9592
Epoch 16/2000
5000/5000 [==============================] - 2s 306us/sample - loss: 0.2500 - f1: 0.9592 - val_loss: 0.2808 - val_f1: 0.9605
Epoch 17/2000
5000/5000 [==============================] - 2s 306us/sample - loss: 0.2780 - f1: 0.9546 - val_loss: 0.3318 - val_f1: 0.9308
Epoch 18/2000
5000/5000 [================

5000/5000 [==============================] - 2s 322us/sample - loss: 0.8895 - f1: 0.8412 - val_loss: 0.7284 - val_f1: 0.8871
Epoch 3/2000
5000/5000 [==============================] - 2s 321us/sample - loss: 0.6130 - f1: 0.9022 - val_loss: 0.5575 - val_f1: 0.9082
Epoch 4/2000
5000/5000 [==============================] - 2s 323us/sample - loss: 0.5268 - f1: 0.9127 - val_loss: 0.4858 - val_f1: 0.9199
Epoch 5/2000
5000/5000 [==============================] - 2s 323us/sample - loss: 0.4691 - f1: 0.9175 - val_loss: 0.4052 - val_f1: 0.9350
Epoch 6/2000
5000/5000 [==============================] - 2s 321us/sample - loss: 0.4020 - f1: 0.9327 - val_loss: 0.4077 - val_f1: 0.9256
Epoch 7/2000
5000/5000 [==============================] - 2s 322us/sample - loss: 0.4008 - f1: 0.9316 - val_loss: 0.3731 - val_f1: 0.9283
Epoch 8/2000
5000/5000 [==============================] - 2s 323us/sample - loss: 0.3829 - f1: 0.9344 - val_loss: 0.3408 - val_f1: 0.9468
Epoch 9/2000
5000/5000 [=======================

15000/15000 [==============================] - 4s 298us/sample - loss: 0.1826 - f1: 0.9892 - val_loss: 0.1102 - val_f1: 0.9953
Epoch 19/2000
15000/15000 [==============================] - 4s 296us/sample - loss: 0.1621 - f1: 0.9906 - val_loss: 0.2353 - val_f1: 0.9799
Epoch 20/2000
15000/15000 [==============================] - 5s 300us/sample - loss: 0.1336 - f1: 0.9945 - val_loss: 0.1364 - val_f1: 0.9947
Epoch 21/2000
15000/15000 [==============================] - 4s 297us/sample - loss: 0.1329 - f1: 0.9922 - val_loss: 0.1862 - val_f1: 0.9901
Epoch 22/2000
15000/15000 [==============================] - 8s 505us/sample - loss: 0.1154 - f1: 0.9960 - val_loss: 0.1363 - val_f1: 0.9946
Running through fold 4
Train on 15000 samples, validate on 3000 samples
Epoch 1/2000
15000/15000 [==============================] - 8s 504us/sample - loss: 1.1815 - f1: 0.7283 - val_loss: 0.5105 - val_f1: 0.9133
Epoch 2/2000
15000/15000 [==============================] - 4s 297us/sample - loss: 0.4483 - f1: 